In [ ]:
import numpy as np
import holodeck as holo
from holodeck.constants import YR, GYR
from holodeck import utils, cosmo
import holodeck.sam_cython

import tqdm

SHAPE = 100
TIME = 3 * GYR

DEF_NUM_FBINS = 7
DEF_PTA_DUR = 16.03     # [yrs]
SHAPE = 100

# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs_cents = utils.midpoints(fobs_edges)
fobs_orb_cents = fobs_cents / 2.0


sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)

# hard = holo.hardening.Hard_GW
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, TIME)

redz_final, diff_num = holodeck.sam_cython.dynamic_binary_number_at_fobs(fobs_orb_cents, sam, hard, cosmo)

# cy_norm = holodeck.sam_cython.find_2pl_hardening_norm(TIME, sam, hard, 10)

# Solving for Hardening Rate

In [ ]:
import numpy as np
import holodeck as holo
from holodeck.constants import YR, GYR, PC
from holodeck import utils, cosmo
import holodeck.sam_cython

import tqdm

SHAPE = (12, 13, 14)
TIME = 3 * GYR
SEPA_INIT = 1e3 * PC
RCHAR = 10.0 * PC
GAMMA_INNER = -1.0
GAMMA_OUTER = +1.5

sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)

old_hard = holo.hardening.Fixed_Time_2PL.from_sam(
    sam, TIME,
    sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER
)
old_norm = old_hard._norm.reshape(sam.shape)[:, :, 0]
print(old_norm.shape)

new_hard = holo.hardening.Fixed_Time_2PL_SAM(
    sam, TIME,
    sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER
)
new_norm = new_hard._norm
print(new_norm.shape)

In [ ]:
print(utils.stats(old_norm))
print(utils.stats(new_norm))
print(utils.stats(new_norm/old_norm))

In [ ]:
print(old_norm[0, :])
print(new_norm[0, :])
print()
print(old_norm[:, 5])
print(new_norm[:, 5])

# Test Binary Evolution

In [ ]:
print("Hello")

import numpy as np
import holodeck as holo
from datetime import datetime
import holodeck.sam_cython
from holodeck.constants import YR, GYR, PC
from holodeck import utils, cosmo

DEF_NUM_FBINS = 7
DEF_PTA_DUR = 16.03     # [yrs]
SHAPE = 100
TIME = 3 * GYR

SEPA_INIT = 1e3 * PC
RCHAR = 10.0 * PC
GAMMA_INNER = -1.0
GAMMA_OUTER = +1.5


# Choose observed GW-Frequency bins based on nyquist sampling
fobs_edges = utils.nyquist_freqs_edges(DEF_PTA_DUR*YR, cad=0.1*YR)
fobs_edges = fobs_edges[:DEF_NUM_FBINS+1]
fobs_cents = utils.midpoints(fobs_edges)
fobs_orb_cents = fobs_cents / 2.0

sam = holo.sam.Semi_Analytic_Model(shape=SHAPE)

new_hard = holo.hardening.Fixed_Time_2PL_SAM(
    sam, TIME,
    sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER
)

new_redz_final, new_dnum = holodeck.sam_cython.dynamic_binary_number_at_fobs(fobs_orb_cents, sam, new_hard, cosmo)

# old_hard = holo.hardening.Fixed_Time_2PL.from_sam(
#     sam, TIME,
#     sepa_init=SEPA_INIT, rchar=RCHAR, gamma_inner=GAMMA_INNER, gamma_outer=GAMMA_OUTER
# )

# edges, old_dnum = sam.dynamic_binary_number(old_hard, fobs_orb_cents, zero_stalled=True, zero_coalesced=True)
# old_redz_final = sam._redz_final

In [ ]:
sel1 = (new_redz_final > 0.0)
sel2 = (old_redz_final > 0.0)
print(utils.stats(new_redz_final[sel1]))
print(utils.stats(old_redz_final[sel2]))
sel = sel1 & sel2
print(utils.frac_str(sel1))
print(utils.frac_str(sel2))
print(utils.frac_str(sel))
print(utils.stats(new_redz_final[sel]/old_redz_final[sel]))

In [ ]:
print(utils.stats(dnum))
print(utils.stats(diff_num))
sel1 = (diff_num > 0.0)
sel2 = (dnum > 0.0)
sel = sel1 & sel2
print(utils.frac_str(sel1))
print(utils.frac_str(sel2))
print(utils.frac_str(sel))
print(utils.stats(diff_num[sel]/dnum[sel]))

In [ ]:
fobs_orb_cents

In [ ]:
bads = (sel1 & ~sel2) | (~sel1 & sel2)
# bads = (sel1 & ~sel2)
# bads = (~sel1 & sel2)
print(utils.frac_str(bads))
bads = np.where(bads)

fig, axes = plt.subplots(figsize=[10, 10], nrows=4)
for ii, (ax, bb) in enumerate(zip(axes, bads)):
    ax.set(xscale='log', yscale='log')
    ax.hist(edges[ii][bb], bins=edges[ii], density=False)
    
plt.show()

In [ ]:
cosmo._grid_dcom/MPC